<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/ml_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Импорт**

In [77]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [78]:
from IPython.display import SVG, display, HTML

In [79]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [80]:
import spacy
import numpy as np

import gensim
from gensim.models import Word2Vec 
from gensim.models import FastText

from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [81]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [82]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

annotations.csv       hv-info_gepatit2.txt	 sample_data
annotation_table.csv  neuroleptic_antidepr2.txt
content		      rakpobedim_ru2.txt
MyDrive
.shortcut-targets-by-id
.file-revisions-by-id
.Trash


In [83]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [84]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-23 18:35:34--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt.1’

neuroleptic_antidep 100%[===================>]  11.12M  22.3MB/s    in 0.5s    

2021-05-23 18:35:34 (22.3 MB/s) - ‘neuroleptic_antidepr2.txt.1’ saved [11655409/11655409]

--2021-05-23 18:35:35--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:

In [85]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv

--2021-05-23 18:35:36--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-23 18:35:37 ERROR 404: Not Found.

--2021-05-23 18:35:37--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2045566 (2.0M) [text/plain]
Saving to: ‘annotation_table.csv.1’

annotation_table.cs 100%[===================>]   1.95M  11.2MB/s    in 0.

In [86]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [87]:
posts = my_txt.split('----')

## **Обучение векторизатора**

In [88]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]


sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]
ft = FastText(sentences_tokenized, window=5, min_count=5, workers=4,sg=1)

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [501]:
count = 0
with open('annotation_table.csv') as csvfile:
  with open('annotations.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    for new_row in csvfile:
      count +=1
      # print(i.split('\t'))
      if len(new_row.split('\t')) > 19:
        new_row = new_row.split('\t')[:20]
        new_row[-1] = new_row[-1][:-1] # cut \n
      writer.writerow(new_row)
count

27019

In [502]:
# # with open('coref_df_anns60k.csv') as csvfile:
# with open('new_anns60k.csv') as csvfile:
#   reader = csv.reader(csvfile, delimiter='\t')
#   with open('new_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     c = 0
#     for row in reader:
#       c +=1
#       new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
#       if len(new_row) > 9:
#         new_row = new_row[:9]
#       else:
#         new_row = [0 for i in range(9)]
#       writer.writerow(new_row)
# c


In [503]:
# !rm new_anns60k.csv.1 new_anns60k.csv.2 new_df_anns60k.csv new_anns60k.csv
# !ls

In [504]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('annotations.csv', delimiter='\t')
anns_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27013,и,0,22,68.0,0.0,7.0,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,0,0.0,NaN
27014,таможенные,0,22,68.0,0.0,8.0,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,0,0.0,plur
27015,пошлины,0,22,68.0,0.0,9.0,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,0,0.0,plur
27016,как,0,22,68.0,0.0,10.0,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,0,0.0,NaN


In [505]:
tok_list = []
for token in anns_df.TOKEN:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['TOKEN_VECT'] = tok_list


tok_list = []
for token in anns_df.HEAD:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['HEAD_VECT'] = tok_list

In [192]:
# tok_list = []
# for sent in 
# for token in anns_df.HEAD:
#   try:
#     tok_list.append(ft.wv.__getitem__(token.lower()))
#   except KeyError:
#     tok_list.append(0)
#     print(token, 'here')
# anns_df['TOKEN_VECT'] = tok_list

In [506]:
anns_df = anns_df.loc[(anns_df['TOPIC_NUM'].isin(list(range(30)))) | (anns_df['SENT_NUM'].isin(list(range(10000))))]
anns_df = anns_df.reset_index().drop(['index'], axis=1)

In [507]:
topics = list(set(anns_df.TOPIC_NUM.values))
topics
sent_vect_list = []
# номера предложений топика
for topic_num in topics:

  sents_in_topic = list(set(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num].SENT_NUM))
  topic_len = len(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num])
  for sent_num in sents_in_topic:
    tok_vects = anns_df.loc[(anns_df['TOPIC_NUM'] == topic_num) & (anns_df['SENT_NUM'] == sent_num)].TOKEN_VECT
    try:
      sent_vect = sum(list(tok_vects))/len(tok_vects)
    except ZeroDivisionError:
      sent_vect = 0
      print(anns_df.loc[(anns_df['SENT_NUM'] == sent_num)])
    for i in range(len(tok_vects)):
      sent_vect_list.append(sent_vect)
len(sent_vect_list)
# если не класть нулевых предложений - будет длина датафрейма, как он раньше импортировлся, но поздно.

26497

In [508]:
anns_df['SENT_VECT'] = sent_vect_list

Проверим данные

In [509]:
anns_df.SENT_NUM[18]

1.0

In [197]:
# topics = list(set(df.TOPIC_NUM.values)) 
# for topic_num in topics:
#   # if topic_num == 2:
#   #   break
#   # извлекаем предложения топика
#   sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
#   for sent_num in sents_in_topic:
#     sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num)]

In [198]:
# vvv = 0
# with open('content/MyDrive/coref_fasttext60k_1.csv') as csvfile:
#   with open('new_coref_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     for new_row in csvfile:
#       vvv +=1
#       # print(i.split('\t'))
#       if len(new_row.split('\t')) > 9:
#         new_row = new_row.split('\t')[:9]
#       writer.writerow(new_row)
# vvv

In [199]:
# cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k_1.csv', delimiter=' ')
# cor_df

In [200]:
# cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

In [201]:
# cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]
# cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]
# cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]
# cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]


Добавим размеченные данные, и посмотрим, что получилось.

In [202]:
# cor_df['NER'] = anns_df['NER']
# cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
# cor_df['ANN_TOKEN'] = anns_df['TOKEN']
# cor_df

In [203]:
# cor_df.loc()

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [510]:
df = anns_df.iloc[:15000,:]

Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [511]:
len(df.loc[df['NER'] == 1])

581

Из них местоимений (прономинальные анафоры).

In [512]:
len(df.loc[df['NER'] == 3])

113

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [513]:
len(df.loc[df['NER'] == 2])

215

In [514]:
len(df.loc[df['NER'].isin([1,2,3])])

909

In [515]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [516]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.75968766, 0.7930102, 0.7499509, 0.17975913,...","[-0.69239724, 1.2821122, -0.1410619, 0.6858161...","[0.048771407, 0.44197467, -0.0020826375, -0.12..."
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,"[0.45758146, 0.6532903, 0.57526225, -0.0656477...","[0.75968766, 0.7930102, 0.7499509, 0.17975913,...","[0.048771407, 0.44197467, -0.0020826375, -0.12..."
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,"[0.3839148, 0.8262058, 0.5599211, -0.009722586...","[0.45758146, 0.6532903, 0.57526225, -0.0656477...","[0.048771407, 0.44197467, -0.0020826375, -0.12..."
21,лекарством,0,1,1.0,1.0,10.0,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,2,0.0,sing,"[-0.15582104, 0.6123812, 0.13382685, -0.038209...","[0.19143681, 0.17168443, 0.19007882, 0.2119140...","[0.048771407, 0.44197467, -0.0020826375, -0.12..."
142,рисполепта,0,1,1.0,13.0,7.0,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,1,142.0,sing,"[0.47953546, 0.26583806, 0.55655634, -0.127523...","[0.75288916, 0.99993676, 0.8061505, 0.562115, ...","[0.048771407, 0.44197467, -0.0020826375, -0.12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14807,абраксан,1,15,18.0,3.0,8.0,anim,gent,femn,NaN,NOUN,obj,спросите,NaN,NaN,NaN,VERB,1,2215.0,plur,"[0.58235055, 0.84001696, 0.58783007, 0.1391526...","[-0.36687627, 0.11346465, -0.4831177, -0.53116...","[0.06955772, 0.51524806, 0.12977563, -0.019205..."
14809,гемцитабином,1,15,18.0,3.0,10.0,inan,ablt,masc,NaN,NOUN,nmod,абраксан,anim,gent,femn,NOUN,1,2217.0,sing,"[-0.06577667, 0.8739506, 0.32398385, 0.2274458...","[0.58235055, 0.84001696, 0.58783007, 0.1391526...","[0.06955772, 0.51524806, 0.12977563, -0.019205..."
14856,фолфиринокс,1,15,18.0,8.0,3.0,inan,nomn,masc,NaN,NOUN,nmod,схема,inan,nomn,femn,NOUN,1,2264.0,sing,"[-0.07617579, 0.33528274, 0.22748771, -0.29112...","[0.3198326, 0.22783576, 0.34864527, -0.4475992...","[0.06955772, 0.51524806, 0.12977563, -0.019205..."
14887,иринотекану,1,15,19.0,0.0,12.0,inan,datv,masc,NaN,NOUN,nmod,аннотации,inan,nomn,femn,NOUN,1,2295.0,sing,"[0.11631464, 0.6957289, 0.1002421, 0.062965505...","[0.03523055, 0.4794186, 0.1702597, -0.30880395...","[0.06955772, 0.51524806, 0.12977563, -0.019205..."


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [517]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.75968766, 0.7930102, 0.7499509, 0.17975913,...","[-0.69239724, 1.2821122, -0.1410619, 0.6858161...","[0.048771407, 0.44197467, -0.0020826375, -0.12..."
576,зелдокс,0,2,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.61519486, 0.60658085, 0.519314, 0.06218257,...","[-0.69239724, 1.2821122, -0.1410619, 0.6858161...","[0.019036854, 0.6033597, 0.06678928, 0.0015192..."
1870,оланзапин,0,3,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.55982906, 0.66840506, 1.0353385, -0.051604,...","[-0.69239724, 1.2821122, -0.1410619, 0.6858161...","[-0.0494221, 0.5309699, -0.046369508, 0.076767..."
5555,галоперидол,0,4,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,1,2.0,sing,"[0.5388747, 0.7892336, 0.37869507, -0.1397206,...","[-0.08959604, 0.55968773, -0.63259286, -0.1375...","[0.19108115, 0.49012217, 0.13618204, -0.024763..."
6200,тиаприд,0,5,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,3.0,sing,"[0.5029817, 0.60633594, 0.96002305, 0.14742985...","[-0.69239724, 1.2821122, -0.1410619, 0.6858161...","[-0.020655425, 0.6038477, 0.15054955, -0.11636..."
8450,сероквель,0,6,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.43370357, 0.6534007, 0.9349229, 0.18017675,...","[-0.69239724, 1.2821122, -0.1410619, 0.6858161...","[-0.10070752, 0.6425655, -0.292019, -0.1921949..."
8541,день,0,7,0.0,0.0,2.0,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,0,0.0,sing,"[-0.29981008, 0.4411791, -1.3673569, -0.345929...","[-0.29981008, 0.4411791, -1.3673569, -0.345929...","[-0.21634302, 0.29559392, -0.20610677, -0.0527..."
11117,500,0,8,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,0,0.0,NaN,"[-0.3778276, 0.5258853, 0.19279899, 0.37503105...","[0.4394574, 0.38244852, 0.7334229, 0.047853086...","[0.108294666, 0.3999989, 0.07735039, -0.098605..."
11787,капецитабин,0,10,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,1,2.0,sing,"[0.482059, 0.39778724, 0.46172807, 0.2793771, ...","[0.482059, 0.39778724, 0.46172807, 0.2793771, ...","[0.108294666, 0.3999989, 0.07735039, -0.098605..."
11806,иресса,0,11,0.0,0.0,2.0,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,1,2.0,NaN,"[0.43822294, 0.33076608, -0.09914217, -0.40014...","[1.2610537, 0.23178987, 0.08240616, 0.47324196...","[0.108294666, 0.3999989, 0.07735039, -0.098605..."


In [518]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [519]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
# head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [520]:
df.TOKEN_VECT[710]

0

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [521]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [522]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
# make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [523]:
df.TOKEN[1]

'препарат'

In [524]:
# df['SUM_VECT']= [(df.TOKEN_VECT[i] + df.SENT_VECT[i])/3 for i in range(len(df.SENT_VECT))]

Посмотрим на обработанные данные.

In [525]:
df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,0.496000,0.514374,0.991549
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing,0.534183,0.514374,0.991549
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,0.796717,0.879256,0.991549
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,0.816754,0.591218,0.991549
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,0.721748,0.651754,0.991549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,эта,0,16,3.0,0.0,10.0,NaN,nomn,femn,NaN,ADJF,det,вакцина,inan,nomn,femn,NOUN,0,0.0,sing,0.390272,0.494690,0.982792
14996,вакцина,0,16,3.0,0.0,11.0,inan,nomn,femn,NaN,NOUN,nsubj,аукнется,NaN,NaN,NaN,VERB,0,0.0,sing,0.566636,0.560179,0.982792
14997,аукнется,0,16,3.0,0.0,12.0,NaN,NaN,NaN,3per,VERB,csubj,неизвестно,NaN,NaN,NaN,ADVB,0,0.0,sing,0.526535,0.520202,0.982792
14998,на,0,16,3.0,0.0,13.0,NaN,NaN,NaN,NaN,PREP,case,организме,inan,loct,masc,NOUN,0,0.0,NaN,0.444369,0.456697,0.982792


# **ПРАВИЛОВЫЙ АЛГОРИТМ**

In [566]:
df.SENT_NUM = [int(i) for i in df['SENT_NUM']]
df.POST_NUM = [int(i) for i in df['POST_NUM']]
# df.TOKEN_NUM = [float(i) for i in df['TOKEN_NUM']]
df.TOPIC_NUM = [int(i) for i in df['TOPIC_NUM']]
df.NER = [int(i) for i in df['NER']]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [535]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [536]:
type(df['SENT_NUM'][18])

numpy.int64

In [553]:

topic_num = 1
topic_df = df.loc[df['TOPIC_NUM'] == topic_num]
# номера предложений топика
sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
sents_in_topic
# достать пост
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1)]
# достать конкретное предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)]
# достать токен из предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)]


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
5,не,0,1,1,0,0.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,0.424626,0.483825,0.991549
6,знаю,0,1,1,0,1.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.525478,0.483825,0.991549
7,стоило,0,1,1,0,2.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.471137,0.483825,0.991549
8,ли,0,1,1,0,3.0,NaN,NaN,NaN,NaN,CONJ,advmod,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.463352,0.449312,0.991549
9,мне,0,1,1,0,4.0,NaN,datv,NaN,1per,NPRO,iobj,стоило,NaN,NaN,neut,VERB,0,0.0,sing,0.529644,0.449312,0.991549
10,писать,0,1,1,0,5.0,NaN,NaN,NaN,NaN,INFN,csubj,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.471329,0.449312,0.991549


In [551]:
for i in df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)].values:
  print(i[-5])

0.0
0.0
0.0
0.0
0.0
0.0


In [548]:
# анафоры
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)
& (df['NER'] > 1)].values[0]

array(['лекарством', '0', 1, 1, 1, 10, 'inan', 'ablt', 'neut', nan,
       'NOUN', 'conj', 'нет', nan, nan, nan, 'PRED', 2, 0.0, 'sing',
       0.41497814655303955, 0.5225167870521545, 0.9915487766265869],
      dtype=object)

In [540]:
# sents_in_topic 
# topic_num
# df = df.reset_index()

1

In [559]:
post = df.loc[(df['TOPIC_NUM'] == topic_num) \
              & (df['POST_NUM'] == post_num) \
              & (df['SENT_NUM'] <= sent_num)]

# current_antecedents = post.loc[(sent['NER'] == 1) ]

In [ ]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [576]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  if post_num == 3:
    break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))
  print(posts_in_topic, 'posts in top\n')

  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    if post_num == 3:
      break

    for sent_num in sents_in_post:
      # print(sents_in_post,'sents in post')
      # print(topic_num  , 'topic num')
      # print(post_num , 'post num')
      # print(sent_num, 'sent_num\n')
      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      current_anaphors = sent.loc[(sent['NER'] > 1)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]

      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) == 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            # в противном случае присваиваем нулевой класс
            ann_clusters.append(anfr[-5])
            # и класс, который определило правило(антецедента из предложения)
            my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      if len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num) \
                        & (df['POST_NUM'] <= post_num) \
                        & (df['SENT_NUM'] <= sent_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]

          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            if anfr[-4] == current_antecedents.NUMBER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)

        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])
              print(ann_clusters, my_clusters, 'well well')
              print('----')
              print(current_antecedents.values)
              print('----')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
      print('---')
      print(ann_clusters, my_clusters)
      print('---')
      if sent_num == 5:
        break




[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] posts in top

[0.0] [2.0] am I here?
['лекарством'] ['лекарством'] [0.0]
['рисперидон', 'рисполепт', 'сперидан'] ['рисперидон', 'рисполепт', 'сперидан'] [2.0, 3.0, 4.0]
---
[0.0] [2.0]
---
['таблетки'] ['таблетки'] [0.0]
['рисполепта'] ['рисполепта'] [142.0]
---
[0.0, 0.0] [2.0, 0.0]
---
['таблетки'] ['таблетки'] [0.0]
['рисперидон', 'рисполепт', 'сперидан'] ['рисперидон', 'рисполепт', 'сперидан'] [2.0, 3.0, 4.0]
---
[0.0, 0.0, 0.0] [2.0, 0.0, 0.0]
---
[0.0, 0.0, 0.0, 344.0] [2.0, 0.0, 0.0, 11.0] well well
----
[['рисполепта' '1' 1 2 4 11.0 'inan' 'nomn' 'femn' nan 'NOUN' 'nmod'
  'приеме' 'inan' 'loct' 'masc' 'NOUN' 1 344.0 'sing' 0.7454516887664795
  0.4778185188770294 0.9830769300460815]]
----
['препарата'] ['препарата'] [344.0]
['рисполепта'] ['рисполепта'] [344.0]
---
[0.0, 0.0, 0.0, 344.0] [2.0, 0.0, 0.0, 11.0]
---


In [227]:
df.loc[df['SENT_NUM'] != 0.0]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,TOKEN_VECT,HEAD_VECT,SENT_VECT
11,у,0,1,1,1,0,NaN,NaN,NaN,NaN,PREP,case,меня,NaN,accs,NaN,NPRO,0,0.0,0.487304,0.399725,0.991549
12,меня,0,1,1,1,1,NaN,accs,NaN,1per,NPRO,obl,нет,NaN,NaN,NaN,PRED,0,0.0,0.448140,0.522517,0.991549
13,практически,0,1,1,1,2,NaN,NaN,NaN,NaN,ADVB,advmod,нет,NaN,NaN,NaN,PRED,0,0.0,0.509971,0.522517,0.991549
14,нет,0,1,1,1,3,NaN,NaN,NaN,NaN,PRED,ROOT,нет,NaN,NaN,NaN,PRED,0,0.0,0.490999,0.522517,0.991549
15,никаких,0,1,1,1,4,NaN,gent,NaN,NaN,ADJF,det,проблем,inan,gent,femn,NOUN,0,0.0,0.400735,0.553550,0.991549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14972,только,0,16,1,5,3,NaN,NaN,NaN,NaN,ADVB,advmod,температура,inan,nomn,femn,NOUN,0,0.0,0.443920,0.324845,0.982792
14973,температура,0,16,1,5,4,inan,nomn,femn,NaN,NOUN,nsubj,была,NaN,NaN,femn,VERB,0,0.0,0.410007,0.428408,0.982792
14974,после,0,16,1,5,5,NaN,NaN,NaN,NaN,PREP,case,прививки,inan,gent,femn,NOUN,0,0.0,0.435863,0.523618,0.982792
14975,второй,0,16,1,5,6,NaN,gent,femn,NaN,ADJF,amod,прививки,inan,gent,femn,NOUN,0,0.0,0.528366,0.523618,0.982792


In [228]:
for i in range(len(df)):
  current_raw = df.iloc[[i]].loc[df['NER'] != 0.0]
  print(current_raw)
  if i == 3:
    break

Empty DataFrame
Columns: [TOKEN, IS_ANSWER, TOPIC_NUM, POST_NUM, SENT_NUM, TOKEN_NUM, ANIMACY, CASE, GENDER, PERSON, POS, DEPENDENCY, HEAD, HEAD_ANIMACY, HEAD_CASE, HEAD_GENDER, HEAD_POS, NER, COREFERENCE_CLUSTER, TOKEN_VECT, HEAD_VECT, SENT_VECT]
Index: []
Empty DataFrame
Columns: [TOKEN, IS_ANSWER, TOPIC_NUM, POST_NUM, SENT_NUM, TOKEN_NUM, ANIMACY, CASE, GENDER, PERSON, POS, DEPENDENCY, HEAD, HEAD_ANIMACY, HEAD_CASE, HEAD_GENDER, HEAD_POS, NER, COREFERENCE_CLUSTER, TOKEN_VECT, HEAD_VECT, SENT_VECT]
Index: []
        TOKEN IS_ANSWER  TOPIC_NUM  ...  TOKEN_VECT  HEAD_VECT  SENT_VECT
2  рисперидон         0          1  ...    0.796717   0.879256   0.991549

[1 rows x 22 columns]
       TOKEN IS_ANSWER  TOPIC_NUM  ...  TOKEN_VECT  HEAD_VECT  SENT_VECT
3  рисполепт         0          1  ...    0.816754   0.591218   0.991549

[1 rows x 22 columns]


# **МОДЕЛИ.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [229]:
for_factorize = 'ANIMACY	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS']

In [230]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Index(['inan', 'anim'], dtype='object')


array([-1,  0,  1, ..., -1, -1,  0])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [231]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2', 'voct', 'gen2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl', 'csubj:pass'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'привился', 'прививки', 'решать', 'неизвестно', 'быстро', 'проводились',
        'испытания', 'аукнется', 'вакцина', 'организме'],
       dtype='object', length=2514),
 'HEAD_ANIMACY': Index(['inan', 'ani

In [ ]:

# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [232]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0,0,2,1,0,0,-1,1,2,1,0,0,0,1,1,2.0,0.796717,0.879256,0.991549
3,рисполепт,0,1,0,0,3,0,1,1,-1,1,3,2,1,0,0,1,1,3.0,0.816754,0.591218,0.991549
4,сперидан,0,1,0,0,4,-1,-1,0,-1,2,4,3,0,1,1,1,1,4.0,0.721748,0.651754,0.991549
21,лекарством,0,1,1,1,10,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,0.414978,0.522517,0.991549
142,рисполепта,0,1,1,13,7,0,0,1,-1,1,21,69,0,2,1,1,1,142.0,0.745452,0.465865,0.991549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14759,hifu,1,15,17,12,4,-1,-1,-1,-1,-1,1,1903,-1,-1,-1,0,1,2167.0,0.515703,0.520144,0.983690
14807,абраксан,1,15,18,3,8,1,1,1,-1,1,1,2480,-1,-1,-1,0,1,2215.0,0.705895,0.414157,0.983690
14809,гемцитабином,1,15,18,3,10,0,5,0,-1,1,21,2470,1,1,1,1,1,2217.0,0.578017,0.545478,0.983690
14856,фолфиринокс,1,15,18,8,3,0,0,0,-1,1,21,1910,0,0,1,1,1,2264.0,0.725166,0.452210,0.983690


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [359]:
# train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)
train, test = train_test_split(df.loc[(df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)

In [360]:
train = train.reset_index()

In [361]:
# train = train.iloc[:,:24].drop(['TOKEN', 'level_0', 'index'], axis=1)
# train = train.drop(['TOKEN', 'index'], axis=1)

In [362]:
test = test.reset_index()

In [363]:
test.head(5)

,index,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,6074,лекарство,1,4,7,7,15,0,0,2,-1,1,1,1158,-1,-1,-1,0,2,449.0,0.425743,0.462535,0.884257
1,7618,нейролептикам,0,5,39,1,5,0,2,0,-1,1,21,1513,1,0,0,1,2,0.0,0.677822,0.251495,0.933704
2,4605,препарат,0,3,31,2,7,0,0,0,-1,1,1,73,-1,-1,-1,0,2,2711.0,0.534183,0.559538,0.953468
3,728,препарата,1,2,3,5,6,0,1,0,-1,1,21,292,0,5,0,1,2,108.0,0.473595,0.515444,0.965674
4,14367,препарату,1,15,13,4,3,0,2,0,-1,1,21,1419,0,4,1,1,2,1757.0,0.521242,0.426579,0.986330


In [364]:
# test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)
# test = test.drop(['TOKEN', 'index'], axis=1)

In [365]:
# test.loc[test['NER'] != 0]

In [366]:
# ['TOPIC_NUM',	'POST_NUM',	'TOKEN_NUM',	'GENDER',	'PERSON', 'TOKEN', 'TOKEN_VECT', 'IS_ANSWER', 'SENT_NUM', 'HEAD_VECT', 'HEAD_CHILDS_VECT', 
#  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',	'POS',	'DEPENDENCY',	'HEAD',	'HEAD_POS']

In [367]:
# best set

# X, y, X_test, y_test = train.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',				'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
# train.COREFERENCE_CLUSTER, test.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',			'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

Делаем наборы для обучения моделей.

In [421]:
X, y, X_test, y_test = train.drop(['TOKEN','GENDER',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',				'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop(['TOKEN','GENDER',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',			'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

In [422]:
# X, y, X_test, y_test = train.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], train.COREFERENCE_CLUSTER, test.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], test.COREFERENCE_CLUSTER

In [423]:
X

,TOKEN_NUM,POS,NER,COREFERENCE_CLUSTER,TOKEN_VECT,SENT_VECT
0,7,1,2,786.0,0.534183,0.947569
1,21,5,3,1262.0,0.503539,0.960946
2,2,5,3,1455.0,0.484660,0.933704
3,10,1,2,0.0,0.474492,0.972606
4,7,1,2,0.0,0.452211,0.751797
...,...,...,...,...,...,...
257,15,5,3,0.0,0.408631,0.865208
258,6,1,2,0.0,0.529826,0.983690
259,0,5,3,766.0,0.467885,0.924827
260,6,1,2,987.0,0.376815,0.956613


In [424]:
y

0       786.0
1      1262.0
2      1455.0
3         0.0
4         0.0
        ...  
257       0.0
258       0.0
259     766.0
260     987.0
261       0.0
Name: COREFERENCE_CLUSTER, Length: 262, dtype: float64

Обучаем модели.

In [425]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test)

0.6212121212121212

In [426]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

0.6666666666666666

In [427]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.5303030303030303

In [428]:
regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)

1.0

In [430]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.42424242424242425

In [431]:

knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.6363636363636364

In [432]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


# **ОЦЕНКА МОДЕЛЕЙ**

In [433]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [434]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [435]:
# my_eval(list(naive_pred), list(y_test))

In [436]:
my_eval(list(pred_knn), list(y_test))

18  true positive
19  false positive
0  true negative
0  false negative
---------
0.4864864864864865  presicion
1.0  recall
0.6545454545454547  f-score


In [437]:
my_eval(list(pred_entropy), list(y_test))

20  true positive
21  false positive
0  true negative
0  false negative
---------
0.4878048780487805  presicion
1.0  recall
0.6557377049180327  f-score


In [438]:
my_eval(list(pred), list(y_test))

17  true positive
24  false positive
0  true negative
0  false negative
---------
0.4146341463414634  presicion
1.0  recall
0.5862068965517241  f-score


In [439]:
my_eval(list(pred_randf), list(y_test))

11  true positive
28  false positive
0  true negative
0  false negative
---------
0.28205128205128205  presicion
1.0  recall
0.44000000000000006  f-score


In [440]:
my_eval(list(pred_regr), list(y_test))

0  true positive
66  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [441]:
my_eval(list(pred_svm), list(y_test))

4  true positive
20  false positive
0  true negative
0  false negative
---------
0.16666666666666666  presicion
1.0  recall
0.2857142857142857  f-score
